In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [14]:
house_data_bristol = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/house_pricing_bristol 22.42.45.csv")
house_data_cornwall = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/house_pricing_cornwall 22.42.45.csv")
broadband_bristol = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/bristol-broadband-cleaned.csv")
broadband_cornwall = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/cornwall-broadband-cleaned.csv")
crime_bristol = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/bristol-crime-cleaned.csv")
crime_cornwall = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/cornwall-crime-cleaned.csv")
schools_bristol = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/bristol_schools_cleaned.csv")
schools_cornwall = pd.read_csv("/Users/anukul/Desktop/Anukul/Learning/DataScience/Cleaning/cleaned-data/cornwall_schools_cleaned.csv")

In [15]:
def rename_columns(df, rename_dict):
    return df.rename(columns=rename_dict)


In [16]:
crime_bristol = rename_columns(crime_bristol, {"postcode": "Postcode", "city": "Town"})
crime_cornwall = rename_columns(crime_cornwall, {"postcode": "Postcode", "city": "Town"})
house_data_bristol = rename_columns(house_data_bristol, {"Postcode": "Postcode", "Town_City": "Town"})
house_data_cornwall = rename_columns(house_data_cornwall, {"Postcode": "Postcode", "Town_City": "Town"})
broadband_bristol = rename_columns(broadband_bristol, {"postcode_space": "Postcode", "county": "Town"})
broadband_cornwall = rename_columns(broadband_cornwall, {"postcode_space": "Postcode", "county": "Town"})
schools_bristol = rename_columns(schools_bristol, {"Postcode": "Postcode", "Town": "Town"})
schools_cornwall = rename_columns(schools_cornwall, {"Postcode": "Postcode", "Town": "Town"})

Combining dataset for bristol and cornwall

In [19]:
def combine_datasets(*dfs):
    return pd.concat(dfs, join="outer", keys=["Town", "Postcode"])


In [20]:
combined_bristol = combine_datasets(house_data_bristol, broadband_bristol, crime_bristol, schools_bristol)

combined_cornwall = combine_datasets(house_data_cornwall, broadband_cornwall, crime_cornwall, schools_cornwall)

/var/folders/jx/pym939rn0_zgvv112pfcjs8r0000gn/T/ipykernel_27145/4191896102.py:2: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  return pd.concat(dfs, join="outer", keys=["Town", "Postcode"])
/var/folders/jx/pym939rn0_zgvv112pfcjs8r0000gn/T/ipykernel_27145/4191896102.py:2: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  return pd.concat(dfs, join="outer", keys=["Town", "Postcode"])


In [22]:
combined_data = pd.concat([combined_bristol, combined_cornwall], ignore_index=True)

In [23]:
combined_data.columns

Index(['ID', 'Housing_Price', 'Year', 'Postcode', 'Type', 'Town', 'District',
       'County', 'Short_Postcode', 'postcode', 'postcode area',
       'Average_download_speed_Mbit_s', 'Maximum_download_speed_Mbit_s',
       'Minimum_download_speed_Mbit_s', 'Average_upload_speed_Mbit_s',
       'Maximum_upload_speed_Mbit_s', 'Minimum_upload_speed_Mbit_s',
       'Average_data_usage_GB', 'All Premises', 'All Matched Premises',
       'lsoa_area'],
      dtype='object')

Function to calculate a final score based on crime rate, school quality, and house prices

In [30]:
crime_cornwall.columns

Index(['Crime_ID', 'Year', 'LSOA_code', 'LSOA_name', 'Crime_type', 'Postcode',
       'Town'],
      dtype='object')

In [28]:
def calculate_final_score(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    df['Housing_Price'] = df['Housing_Price'].fillna(df['Housing_Price'].median())
    df['crime_rate'] = df['Crime_ID'].notna().astype(int)
    df['school_quality'] = df['OFSTEDRATING'].fillna(0).astype(float)
    
    df['crime_rate_score'] = 10 * (1 - scaler.fit_transform(df[['crime_rate']]))
    df['school_quality_score'] = 10 * scaler.fit_transform(df[['school_quality']])
    df['house_price_score'] = 10 * (1 - scaler.fit_transform(df[['Housing_Price']]))
    
    df['final_score'] = (
        0.4 * df['crime_rate_score'] + 
        0.3 * df['school_quality_score'] + 
        0.3 * df['house_price_score']
    )
    
    return df